In [1]:


def mapMethytilatedCitosine(metfile,intersection_file_path,output_file_path):
    chrs_data = {}
    with gzip.open(metfile, 'rt') as f:
        for line in f:
            chr_index = line.split('\t')[0]
            coordinate_index = int(line.split('\t')[1])
            try:
                chrs_data[chr_index][coordinate_index] = line[:-5]
            except KeyError:
                chrs_data[chr_index] = {coordinate_index:line[:-5]}

    with open(intersection_file_path) as f:
        reader = csv.reader(f, delimiter='\t')
        data = list(reader)

    with open(output_file_path, 'w') as whateverBox:
        for each_box in data:
            box_areas = {}
            box_start = int(each_box[1])
            box_end = int(each_box[2])
            chr_location = each_box[0]
            sequence_identifier = each_box[3]
            # tenemos que aplicar rango desde una posicion inicial porque es un archivo 0 based, si fuese 1 based el rango estaría mal y tendria que restarle
            # uno al archivo al box start
            box_areas['box'] = list(range(box_start, box_end))

            for boxarea in box_areas:
                for coordinate in box_areas[boxarea]:
                    matchline = chrs_data[chr_location].get(coordinate, False)
                    if matchline:
                        whateverBox.write('{}\t{}\t{}\n'.format(matchline, sequence_identifier, boxarea))
                        

def loadFileRemoveMtsAndLowMetReport(replicate,cgcontext):

        bxdf = pd.read_csv(
            replicate,
            sep='\t', 
            names=['chr', 'position', 'strand', 'mets', 'nomets', 'context','sequence'],
            usecols=['chr', 'position','strand', 'context','mets', 'nomets']
            ,dtype={'chr':'str'}, 
            compression='gzip'
                )
        
        bxdf.drop(bxdf[(bxdf.chr == 'Mt') | (bxdf.chr == 'Pt')].index, inplace=True)
        if cgcontext == 'CG':
            bxdf.drop(bxdf[(bxdf.context != 'CG')].index, inplace=True)
        if cgcontext == 'nonCG':
            bxdf.drop(bxdf[(bxdf.context == 'CG')].index, inplace=True)
        
        bxdf = bxdf[bxdf['mets'] + bxdf['nomets'] > 4]
        
        return bxdf

def calculateValueAsPercentageOfMetCs(replicatedf,replicanamedf):
    replicatedf[replicanamedf] = replicatedf['mets'] / (replicatedf['nomets'] + replicatedf['mets'])
    replicatedf = replicatedf.drop(columns=['mets','nomets'])
    return replicatedf
def calculateValueAsWheigtedPercentageOfMetCs(replicatedf=None,replicanamedf=None):
    # replicatedf = replicatedf.drop(columns=['chr','context'])
    # replicatedf = replicatedf.groupby('boxID', as_index=False).sum()
    replicatedf[replicanamedf] = replicatedf['mets'] / (replicatedf['nomets'] + replicatedf['mets'])
    replicatedf = replicatedf.drop(columns=['mets','nomets'])
    return replicatedf
def calculateValueAsCualitative(replicatedf,replicanamedf):
    """only zero or one, meaning one that there is methylation"""
    replicatedf[replicanamedf] = npwhere(replicatedf['mets']>replicatedf['nomets'], 1, 0)
    replicatedf = replicatedf.drop(columns=['mets','nomets'])
    return replicatedf

In [2]:
import csv
import gzip
import time
import os
import pandas as pd
import re
from numpy import where as npwhere
from functools import reduce
working_folder = '/home/joaquin/projects/methylation/data/bisulfite_quick_and_dirty_rep1_rep2'
intersection_file = '/home/joaquin/projects/methylation/data/commonData/arabidopsisThaliana/intersect/Peaks_centered_56bp.bed'
basedmrfolder = '/home/joaquin/projects/methylation/methylation_jose/met_site'
experimentsClasification = {}
for root, dirs, files in os.walk(working_folder):

    for file in files:
        if file.endswith('CX_report.txt.gz'):
            replicate, hour, condition =root.strip().split('/')[-3:]
            experimentName = hour+condition
            experimentsClasification.setdefault(experimentName, [])
            targetMetFilename = root+'/'+file
            experimentsClasification[experimentName].append(targetMetFilename)
            

In [3]:

toplot_mean = []

for metcontext in ['CG']:  
    allthedata = []
    cgdataframe = []
    noncgdataframes = []
    finaldf = None
    for experimentCondition in experimentsClasification:
        listOfDfReplicatesPercentageOfMetCs = []
        listOfDfReplicatesCualitative = []
        listOfNamesReplicates = []
        df_merged = None
        for replicatePath in experimentsClasification[experimentCondition]:
            replicateDf = None
            replicaname =  experimentCondition+replicatePath.split('/')[-4]
            print(replicaname)
            replicateDf = loadFileRemoveMtsAndLowMetReport(replicatePath, metcontext)
            replicateDfPercentageOfMetCs = calculateValueAsPercentageOfMetCs(replicateDf,replicaname)

            listOfNamesReplicates.append(replicaname)
            listOfDfReplicatesPercentageOfMetCs.append(replicateDfPercentageOfMetCs)

        df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['chr', 'position','strand','context',],
                                                    how='outer'), listOfDfReplicatesPercentageOfMetCs)

        df_merged[experimentCondition] = df_merged[listOfNamesReplicates].mean(axis=1)

        df_merged = df_merged.drop(columns=listOfNamesReplicates)
        allthedata.append(df_merged)

    finaldf = reduce(lambda  left,right: pd.merge(left,right,on=['chr', 'position','strand','context'],
                                                how='outer'), allthedata)
    # finaldf = peaks.merge(finaldf, on='boxID', how='outer')
    finaldf = finaldf.sort_values(['chr','position'])[['chr','position','1Mock','1JA','1ACC','6Mock','6JA','6ACC','24Mock','24JA','24ACC']]



    finaldf.to_csv(f'/home/joaquin/projects/methylation/data/bisulfite_quick_and_dirty_rep1_rep2/{metcontext}_BSMet_all_the_samples_no_intersect_files_CG_context.tsv',sep='\t', index=False)
    
    
    # finaldf = finaldf.dropna()


6ACCrep3
6ACCrep1
6ACCrep2
6Mockrep3
6Mockrep1
6Mockrep2
6JArep3
6JArep1
6JArep2
24ACCrep3
24ACCrep1
24ACCrep2
24Mockrep3
24Mockrep1
24Mockrep2
24JArep3
24JArep1
24JArep2
1ACCrep3
1ACCrep1
1ACCrep2
1Mockrep3
1Mockrep1
1Mockrep2
1JArep3
1JArep1
1JArep2


In [10]:
import subprocess

output_folder_path = '/home/joaquin/projects/methylation/data/bisulfite_quick_and_dirty_rep1_rep2/bigwig_from_individual_mets'
chr_sizes = '/home/joaquin/projects/methylation/data/commonData/arabidopsisThaliana/genome.index.txt'

def bedgraphToBwFromMean(bedgraphFilePath, speciesIndexChrSize):
    subprocess.run('bedGraphToBigWig ' + bedgraphFilePath + ' ' + speciesIndexChrSize + ' ' + bedgraphFilePath[:-8]+'bw',
            shell=True)




file_paths = ['CG_BSMet_all_the_samples_no_intersect_files_CG_context','nonCG_BSMet_all_the_samples_no_intersect_files']
context_case_list = ['CG','nonCG']
for file_name, context_case in zip(file_paths, context_case_list):
    df_load = None
    df_load = pd.read_csv(f'/home/joaquin/projects/methylation/data/bisulfite_quick_and_dirty_rep1_rep2/{file_name}.tsv',sep='\t')
    for case in ['1Mock','1JA','1ACC','6Mock','6JA','6ACC','24Mock','24JA','24ACC']:
        bedgraph_file_path = f'{output_folder_path}/{case}_{context_case}.bedgraph'
        subdf = df_load[['chr','position',case]].copy()
        subdf['end'] = subdf['position']
        subdf[['chr','position','end',case]].to_csv(bedgraph_file_path,sep='\t', index=False, header=False)

In [4]:
import subprocess
bedgraphToBwFromMean('/home/joaquin/projects/methylation/data/bisulfite_quick_and_dirty_rep1_rep2/bigwig_from_individual_mets/1Mock_nonCG.bedgraph',chr_sizes)

In [11]:
subdf[['chr','position','end',case]]

,chr,position,end,1Mock
0,1,0,0,0.428571
1,1,1,1,0.428571
2,1,2,2,0.571429
3,1,7,7,0.428571
4,1,8,8,0.428571
...,...,...,...,...
36640674,5,26975483,26975483,0.000000
36640675,5,26975489,26975489,0.200000
36640676,5,26975490,26975490,0.000000
36640677,5,26975491,26975491,0.000000


In [8]:
finaldf = finaldf.sort_values(['chr','position'])[['chr','position','1Mock','1JA','1ACC','6Mock','6JA','6ACC','24Mock','24JA','24ACC']]



finaldf.to_csv(f'/home/joaquin/projects/methylation/data/bisulfite_quick_and_dirty_rep1_rep2/{metcontext}_BSMet_all_the_samples_no_intersect_files.tsv',sep='\t', index=False)
    
    

In [9]:
finaldf

,chr,position,1Mock,1JA,1ACC,6Mock,6JA,6ACC,24Mock,24JA,24ACC
36241545,1,0,0.428571,0.622222,NaN,NaN,0.000000,NaN,0.2,0.571429,0.000000
36241546,1,1,0.428571,0.577778,NaN,NaN,0.166667,NaN,0.2,0.571429,0.166667
36241547,1,2,0.571429,0.633333,NaN,NaN,0.166667,NaN,0.8,0.714286,0.666667
34705254,1,7,0.428571,0.472222,NaN,NaN,0.166667,0.833333,0.2,0.428571,0.166667
34705255,1,8,0.428571,0.513889,NaN,NaN,0.166667,0.833333,0.2,0.285714,0.166667
...,...,...,...,...,...,...,...,...,...,...,...
36003206,5,26975483,0.000000,NaN,NaN,0.2,0.166667,NaN,NaN,0.200000,NaN
36003207,5,26975489,0.200000,NaN,NaN,0.4,0.200000,NaN,NaN,0.200000,NaN
36003208,5,26975490,0.000000,NaN,NaN,0.0,0.200000,NaN,NaN,0.200000,NaN
36003209,5,26975491,0.000000,NaN,NaN,0.2,0.200000,NaN,NaN,0.200000,NaN
